<a href="https://colab.research.google.com/github/GabrielaGroenenewegenVanDerWeijden/Neuro-25/blob/main/Decoding_Horizontal_Eye_Position.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is a (python) notebook.
To be able to use it, go through this:


1. in the file menu (top left), click ```open in playground```
3. still in the file menu, click ```save copy in drive```, to make your own personalized and editable copy of this file.
4. edit as you like. If something breaks irreparably, go back to step 1.

More information about jupyter notebooks and colab is here:

0. Learn about how to use google colaboratory [video](https://www.google.com/search?client=opera&q=introduction+to+google+colab&sourceid=opera&ie=UTF-8&oe=UTF-8#kpvalbx=_gYFDX5-jEcv0kwWY_YGgCg113)
1. Have a look at this [example notebook](https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l01c01_introduction_to_colab_and_python.ipynb) that introduces python and colab.

# How to use this tutorial:

You are expected to read this document sequentially, and answer questions you will find in the comments and at the bottom of the document. You will present your answers at our next encounter.

> To edit and to run, go to the menu `File> Save a Copy in Drive` or `File > Open in Playground Mode`

*Tip: open the table of contents, on the icons on the top-right corner menu bar (look like bullet points).*


# Tutorial: Representing Eye Position in Populations of Spiking Neurons

In this [colab](https://colab.research.google.com/notebooks/intro.ipynb) we will use the simulator [NENGO](www.nengo.ai) to implement a neural population of leaky integrate and fire neurons that represents eye position (a continuous variable). This is an ultra simple example for you to **get a sense of what it means for populations of neurons to encode physical quantities**.

We will **create a population of encoding neurons**, which effectively will be basis functions, and will respond monotonically but non-linearly.

The response of these LIF neurons are derived from response properties found  in the NPH and MVN, which have tuning curves for specific eye positions.

We will then **decode the eye position information** from the population of neurons. We will then verify the quality of the decoding and observe how it changes with the size of the neural population and other biophysical properties of the modeled neurons such as membrane time constant.


## Learning Goals

- Learn how to implement a LIF network in Nengo
- Probe and display network inputs and outputs
- Encode an input quantity in a LIF population
- Decode the quantity from these LIF neurons
- Change properties of Networks, LIF neurons and synapses


For reference code snippets check: https://www.nengo.ai/nengo/examples/advanced/nef-summary.html

## Requirements

- [Basic knowledge about integrate and fire neurons](https://lcnwww.epfl.ch/gerstner/SPNM/node26.html)
  - membrane time constant
  - response function
- Useful but not essential: [Basic notions of object oriented programming](https://www.youtube.com/watch?v=pTB0EiLXUC8)

## Resources

- Nengo uses optimal linear decoding to find weights that reconstruct a signal from neuronal activity. [Here is a video I recorded with the analytical derivation of how to find the optimal decoding weights](https://youtu.be/A8Mc_IsVSTE)
- [A nice set of applets to get a sense of models of spiking](http://jackterwilliger.com/biological-neural-networks-part-i-spiking-neurons/). For the purposes herein, you are interested in the activity of integrate and fire neurons.
- [Wulfram Gerstner explains the dynamics of integrate and fire neurons](https://youtu.be/KGxVwJJC9zs?si=prIKnJAUBg5QNVYi).
- [Learn more about the Nengo simulator here](https://www.nengo.ai/nengo/).



## Glossary

- **Input space**: the set of possible values that input can take. For example, the input space of a single photo receptor is the possible luminance values that the photosensor receives.
- **Sensitivity**: the sensitivity of a (typically sensory) neuron measures the modulation of the neuron to some input

## Initialization

In the cells below we install the nengo simulator and import relevant python packages.

In [ ]:
# Install Nengo into the colab via pip
!pip install nengo

In [ ]:
import nengo # our simulator
import pandas as pd # to collect and handle data frames
import matplotlib.pyplot as plt # for plotting
import numpy as np # for numerical / mathematical functions
import seaborn as sns # for nice plotting

In [ ]:
from nengo.dists import Uniform # a nice uniform distribution
from nengo.processes import WhiteSignal # a noise source
from nengo.utils.ensemble import tuning_curves # tuning curves
# from nengo.utils.ipython import hide_input
from nengo.utils.matplotlib import rasterplot # for a convenient way of displaying raster plots

In [ ]:
# to add the plots directly to the jupyter document:
%matplotlib inline

In [ ]:
from nengo.dists import Uniform # a nice uniform distribution

# Modeling Population Encoding in Nengo

A recipe for a model in Nengo is as follows:

1. Create a model with `nengo.Network()` object.
2. Create and insert inputs in the model via `nengo.Node()`.
3. Add a number of neuronal populations via `nengo.Ensembles()`. Here we can also set the properties of our neurons, such as membrane time constant.
4. Connect those ensembles via synapses via `nengo.Connection()`. Here we can also define synaptic properties. It is here as well, that we may want to define the **decoding functions**.
5. Add some **probes** to the model with `nengo.Probe()` to record variables.
6. Add the model to a `nengo.Simulator()` object and run.
7. Plot selected variables using your package of choice (e.g., seaborn, matplotlib).


## 1. Create a model.

This is the **object** that we will populate with inputs, neurons and probes, and that we will simulate.

In [ ]:
model = nengo.Network(label='NPH and VN') # Label is simply a name (string).

# NPH : nucleus prepositus hippoglossi
# VN  : vestibular nucleus


## 2. Create an input:

Nengo's function `Node()` is used to create inputs to networks. [Lambda notation](https://realpython.com/python-lambda/) is often used for the purpose.



- for example, a function that returns a ramping input:

$$ input(t) = t*2-1 $$

is written in nengo as:

> `input = nengo.Node(lambda t: t * 2 - 1)`

- a function that returns 1 during a certain interval.

> `input = nengo.Node(lambda t: 1 0] if t < 0.1 else 0)`

- a function that produces a sine wave with a certain frequency

> `input = nengo.Node(lambda t: sin(2*pi*t)`

- combine the two last statements

`input = nengo.Node(lambda t: sin(2*pi*t) if t > 1 else 0)`

In [ ]:
with model: # to add things to model, we use 'with'
  input = nengo.Node(lambda t: np.sin(2*1*np.pi * t)) # the input

  # you cal also try other functions as input.
  # input = nengo.Node(lambda t: t * 2 - 1)


  input_probe = nengo.Probe(input) # with Probe we record a variable

In order to display our input, we need to run the model (this populates the output variables). We do that by adding our model to `Simulator()` on `model`  and running for `t` seconds.

In [ ]:
with nengo.Simulator(model) as sim:
    sim.run(1.0)

Our model only has an input so far, no neurons. But we can plot it for sanity checking:

In [ ]:
plt.figure()
plt.plot(sim.trange(), sim.data[input_probe], lw=2) #note 'input_probe' is what we created above. lw is line width.
plt.title("Input signal")
plt.xlabel("Time (s)")
plt.xlim(0, 2)

## 3. Create some Neurons that Cover the Input Space

We will add an ensemble that encodes a continuous variable (representing some physical input to sensory neurons such as pressure, or light intensity). We do this by creating a set of neurons. Each of these neurons has a different tuning curve, such that together the neurons 'cover' the entire 'input space'. These tuning curves will have a mostly-linear relationship with an encoded quantity. In NENGO we refer to neurons with tuning curves as  'encoders'.

---

Firing characteristics of the Nucleus Prepositus Hipoglossi and Medial Vestibular Nucleus

**System Specification:**
>
- Encode for horizontal eye position --
  - Humans: 50deg horizontal eye motion (Davson 1990, p.657).
  - In the model we can transform [-25 to 25] to a range between [-1, 1], without loss of generality!
- Average background firing rates in the NPH: 0-150 Hz (Moschovakis 1997)
- Maximum firing rate ~ 300Hz
- Tuning curve sensitivities (mostly linear!) : 0.1 to 7 Hz / deg





---

### Tuning curves as basis functions
We distribute the encoder neurons over the input space by setting neurons with intercepts along the encoded dimension, in this case from [-1 to 1]. We do this with a helper function `aligned()`, which we define below. We will use the output of this function to parameterize our encoding neurons to cover the base space.

Example: Write a fuction called 'aligned', that returns two lists of N values, where:
- **intercepts**: the value at which the neuron starts firing. Equally spaced points in an interval between [-radius, radius], where radius represents the possible values of an input around zero.
- **encoders**: the slope of the tuning curves of the neuron. Here we simply use a vector populated with equal numbers of -1 (off neurons) and 1 (on neurons). The reasons for this choice are explained in Neuroengineering chapter 4).


In [ ]:
def aligned(n_neurons, radius=0.9):
    intercepts = np.linspace(-radius, radius, n_neurons)
    encoders = np.tile([[1], [-1]], (n_neurons // 2, 1))
    intercepts *= encoders[:, 0]
    return intercepts, encoders

#### Warm-up Exercise:
**Call the function** defined above with 8 neurons and a radius of 1 and **inspect the output**.

In [ ]:
# your code here

### Check your answer below

In [ ]:
intercepts, encoders = aligned(8, 1)
print(intercepts)

### Make an IF population of encoders

Below we create an ensemble of 8 IF neurons to encode the 1-dimensional input (e.g., the eye position) according to specified intercepts, encoders and with specified max_rates.

In [ ]:
with model:
    NPH = nengo.Ensemble( #NPH stands for 'nucleus prepositus hipoglossi'
        8, # number of neurons in the ensemble
        dimensions=1, # encoded stimulus dimensions ("capacity")
        intercepts=intercepts,
        max_rates=Uniform(80, 100), # the maximum firing rate of the neurons are drawn from the uniform distribution
        encoders=encoders,
    )

We can plot the tuning curves of our neurons, with the function `tuning_curves` for population (In this case NPH).

In [ ]:
with nengo.Simulator(model) as sim:
    eval_points, activities = tuning_curves(NPH, sim)

plt.figure()
plt.plot(eval_points, activities, lw=2)
plt.xlabel("Input signal")
plt.ylabel("Firing rate (Hz)")

In [ ]:
help(tuning_curves)

### 5. Connect the Inputs with Neurons

We distribute the input to all neurons in the ensemble via `nengo.Connection()`. By default we connect the input to all postsynaptic neurons with equal weights*. Let's also create a probe for the spikes the model is producing.

In [ ]:
with model:
    nengo.Connection(input, NPH)
    NPH_spikes = nengo.Probe(NPH.neurons)

### 5. Run the Network

In [ ]:
with nengo.Simulator(model) as sim:
    sim.run(1)

## 6. How does the activity of the neurons look like??

Now that we have the encoding neurons, the input and the probes, we can run the simulation and check the firing behavior of the neurons.

#### Raster Plot:

In [ ]:
plt.figure(figsize=[15,4])
ax = plt.subplot(1, 1, 1)
rasterplot(sim.trange(), sim.data[NPH_spikes], ax)
ax.set_xlim(0, 1)
ax.set_ylabel("Neuron")
ax.set_xlabel("Time (s)")

#### Membrane Potential

In [ ]:
with model:
  nengo.Connection(input, NPH)
  NPH_spikes = nengo.Probe(NPH.neurons, synapse=0.05)

with nengo.Simulator(model) as sim:
    sim.run(1)

scale = 180
plt.figure()
for i in range(NPH.n_neurons):
    plt.plot(sim.trange(), sim.data[NPH_spikes][:, i] - i * scale)
plt.xlim(0, 1)
plt.ylim(scale * (-NPH.n_neurons + 1), scale)
plt.ylabel("Neuron");
plt.yticks(
    np.arange(scale / 1.8, (-NPH.n_neurons + 1) * scale, -scale), np.arange(NPH.n_neurons)
);

## 7. Check your decoding skills!


Can we decode those spikes and obtain our signal back? A decoder that produces the identity function can be obtained via a simple Probe. To find the decoding weights, the probe of a population uses the encoded value.

In [ ]:
with model:
    NPH_probe = nengo.Probe(NPH, synapse=0.2)
    # 5ms PSC filter (AMPA like)
    # 10ms PSC filter (GABA like)
    # 100ms PSC filter (NMDA like)

simtime = 5 #seconds

with nengo.Simulator(model) as sim:
    sim.run(simtime)

plt.figure()
plt.plot(sim.trange(), sim.data[NPH_probe], label="Decoded estimate")
plt.plot(sim.trange(), sim.data[input_probe], label="Input signal")
plt.legend(loc="best")
plt.xlim(0, simtime)

# Questions and Exercises

**1.(Question)** What significant assumption have we made about the distribution of tuning curves (hint: what is particular about our chosen intercepts / encoders)?

**2. (Exercise).** Our network is rather small. Increase the number of neurons and observe the impact on decoding quality.

**3. (Check).** Verify that the parameters of our network are specified according to the experimental data. Particularly, make sure that the values for individual neuronal tuning curves in the code above are specified as in the system parameters (section 3).

**4. (Experiment)** Is the decoding quality a function of the input oscillation? In other words, does the network encode all frequencies equally well? Change the frequency of the sinusoidal input function and inspect the decoding quality. Are all possible input frequencies decoded equally well? Is there a dependency between quality of decoding and synapse time constant (determined in line 2 in the code block above).

**5. (Challenge).** Create an 'efference copy' from the encoding population. That is: produce another population downstream from the encoding population that conveys the same 'quantity' (same number of dimensions). Now decode the original value from the second population. Try to spot the main differences in the activity of the two populations. Note: this exercise needs you to code a new Model, with an extra 'ensemble'.


# Bonus: Influence of Membrane Time Constant on Signal Reconstruction

Code gracefully scraped from: https://github.com/ctn-waterloo/modelling_ideas/issues/91

In [ ]:
def go(freq, tau_rc, n_neurons=10, tau_probe=0.005, t=1.0, dt=0.001, seed=0):
    with nengo.Network() as model:
        u = nengo.Node(output=lambda t: np.sin(2*np.pi*freq*t))
        x = nengo.Ensemble(n_neurons, 1, seed=seed,
                           neuron_type=nengo.LIF(tau_rc=tau_rc))
        nengo.Connection(u, x, synapse=None)

        p_u = nengo.Probe(u, synapse=tau_probe)
        p_x = nengo.Probe(x, synapse=tau_probe)

    with nengo.Simulator(model, dt=dt, progress_bar=False) as sim:
        sim.run(t, progress_bar=False)

    return nengo.utils.numpy.rmse(sim.data[p_u], sim.data[p_x])

data = []
for seed in range(5):
    for freq in np.linspace(0, 50, 20):
        for tau_rc in [0.001, 0.005, 0.01, 0.02, 0.1]:
            print(freq, tau_rc)
            data.append((freq, tau_rc, seed, go(freq, tau_rc, seed=seed)))
df = pd.DataFrame(data, columns=("Frequency", "tau_rc", "Seed", "RMSE"))

In [ ]:
plt.figure(figsize=[20,20])
for tau_rc in df.tau_rc.unique():
    sns.regplot(data=df[df['tau_rc'] == tau_rc], x_jitter=1.5,
                x="Frequency", y="RMSE", label=str(tau_rc))
plt.legend()
plt.show()

# Example: Encoding a two dimensional vector

In [ ]:
# Setup the envirnment
import numpy as np
import nengo
from nengo.dists import Uniform

model = nengo.Network(label='2D Representation')
with model:
    #Two represent possible two dimensional input values, we choose sin and cos
    sin = nengo.Node(output=np.sin)
    cos = nengo.Node(output=np.cos)

    # Create here an ensemble with 100 LIF neurons which represents a 2-dimensional signal
    x = nengo.Ensemble(100, dimensions=2, max_rates=Uniform(100, 200))

    #Get the neuron encoders
    encoders = x.encoders.sample(100,2)

    # Connecnting input to ensemble
    # The indices in ensemble 'x' define which dimension the input will project to
    nengo.Connection(sin, x[0])
    nengo.Connection(cos, x[1])


#place a probe to record a selected variable
# Q: what changes with different synaptic time constants?
with model:
    probe1 = nengo.Probe(x.neurons, synapse=0.01)

simtime = 5 #seconds
# run the simulator for five seconds
with nengo.Simulator(model) as sim:
    sim.run(simtime)


# plot the decoded 2D variables
plt.figure()
plt.plot(sim.trange(), sim.data[probe1], label="Decoded estimate")
plt.legend(loc="best")
plt.xlim(0, simtime)

# challenge: can you plot the input signal as well?